In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')

: 

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

In [ ]:
path = './data/'
data = pd.read_csv(path +'Cart_IBrA.csv' , sep=';' )['Código']
tickers = list(data.values+'.SA')
tickers.remove('BRAV3.SA')

In [ ]:
price = yf.download( tickers =  tickers, period='2y' , auto_adjust=True )['Close'].dropna()
price.columns = price.columns.str.replace(r'\.SA$', '', regex=True)
data_pc = price.pct_change()
data_pc.tail(2)

In [ ]:
data_std = data_pc.std()*(252**0.5)*100
data_std.tail()

#### decis de volatilidade

In [ ]:
vol_sorted = data_std.sort_values(ascending=False)
top_decile = vol_sorted[:int(len(data_std)*0.1)]
top_decile.plot.bar(figsize=(16,7),color='green',ylim=top_decile.min()*0.9)
plt.title('Volatility Deciles', fontsize=14)
plt.xlabel('Stocks')
plt.ylabel('Annualised Volatility')
plt.show()

In [ ]:
stock_data = price.loc[:,top_decile.index]
stock_data_pc = stock_data.pct_change()

In [ ]:
portfolio = pd.DataFrame()
portfolio['returns'] = stock_data_pc.mean(axis=1)
portfolio['cumreturn'] = (portfolio+1).cumprod()
portfolio = portfolio.dropna()
portfolio.cumreturn.plot(color='b',figsize=(10,7))
plt.ylabel("Portfolio cumulative returns")
plt.show()

<a id='analyse_SMA'></a>
##### Sinal de Trading baseado em Médias Móveis
1. média móvel simples de 10 dia
2. sinal 1 quando o valor da carteira for maior do que a média móvel simples.

In [ ]:
def get_strategy_returns_sma(portfolio):
    # média móvel simples do período 10
    portfolio['sma10'] = portfolio.cumreturn.rolling(window=10).mean()
    # sinal de trading
    portfolio['signal'] = np.where(portfolio.cumreturn > portfolio.sma10,1,0)
    # retornos da estratégia
    portfolio['str_returns'] = portfolio['returns'].shift(-1) * portfolio['signal']
    return portfolio

def plot_signal(portfolio):
    portfolio['cumreturn'].plot(color='blue')    
    plt.fill_between(portfolio.index, portfolio.cumreturn, where=(portfolio.signal==1), facecolor='w', alpha=0.5)      
    plt.legend()
    plt.ylabel('Portfolio cumulative returns',color='b')
    # Plote os valores do sinal    
    portfolio['signal'].plot(secondary_y=True,figsize=(13,4),color='yellow')
    plt.ylabel('Signal',color='y')
    plt.show()


print(' A linha azul indica o retorno acumulado\n A linha amarela o sinal.\n A área sombreada o período em que estamos comprados')
portfolio = get_strategy_returns_sma(portfolio)    
plot_signal(portfolio)    

In [ ]:
# Plote os retornos do portfolio
def plot_returns_dd(portfolio):
    # ----------- Índice de Sharpe ------------------
    sharpe_ratio = np.mean(portfolio.str_returns)/np.std(portfolio.str_returns)*(252**0.5)
    print('O Sharpe ratio é %.2f ' % sharpe_ratio)

    # ----------- Retornos Cumulatvos da Estratégia ------------------
    portfolio['cum_str_returns'] = (portfolio['str_returns']+1).cumprod()
    portfolio['cum_str_returns'].plot(figsize=(10,5), color='green')
    plt.title('Strategy Returns', fontsize=14)
    plt.ylabel('Cumulative returns')
    plt.show()        

    # ----------- Drawdown ------------------    
    # Calcule o máximo de execução
    running_max = np.maximum.accumulate(portfolio['cum_str_returns'].dropna())
    running_max[running_max < 1] = 1
    # Calcule a porcentagem do drawdown
    drawdown = (portfolio['cum_str_returns'])/running_max - 1
    max_dd = drawdown.min()*100
    print('O drawdonw máximo é %.2f' % max_dd)
    # Plote drawdowns
    drawdown.plot(color='r',figsize=(10,5))
    plt.ylabel('Returns')
    plt.fill_between(drawdown.index, drawdown, color='red')
    plt.grid(which="major", color='k', linestyle='-.', linewidth=0.2)
    plt.show()    
    
plot_returns_dd(portfolio)    

##### Sinais de negociação: breakout

1. *Breakout*: valor máximo da carteira dos dois dias anteriores.
2. *Sinal de negociação*: 1 quando o valor da carteira do dia atual for maior que o valor máximo calculado na etapa 1.

In [ ]:
def get_strategy_returns_breakout(portfolio):
    # Calcule os valores do indicador de breakout
    portfolio['high'] = portfolio.cumreturn.rolling(window=3).max()
    # Crie um sinal de trading
    portfolio['signal'] = np.where(portfolio.cumreturn>=portfolio.high,1,0)
    # Calcule os retornos da estratégia
    portfolio['str_returns'] = portfolio['returns'].shift(-1) * portfolio['signal']
    return portfolio

portfolio = get_strategy_returns_breakout(portfolio)
#plot_signal(portfolio)    
plot_returns_dd(portfolio)    

In [ ]:
portfolio.tail(20)

#### Sinais de negociação  média móvel e breakout

In [ ]:
def get_strategy_returns_sma_breakout(portfolio):
    # Calcule a média móvel simples
    sma10 = portfolio.cumreturn > portfolio.cumreturn.rolling(window=10).mean()
    # Calcule os valores do indicador de breakout
    breakout = portfolio.cumreturn>=portfolio.cumreturn.rolling(window=3).max()

    # Crie um sinal de trading
    portfolio['signal'] = np.where(sma10 & breakout,1,0)

    # Calcule os retornos da estratégia
    portfolio['str_returns'] = portfolio['returns'].shift(-1) * portfolio['signal']
    return portfolio

portfolio = get_strategy_returns_sma_breakout(portfolio)
#plot_signal(portfolio)    
plot_returns_dd(portfolio)    